### Import packages

In [1]:
import json
import pickle
import os
import math
import multiprocessing as mp
import random
from random import shuffle, seed
import re
import gzip
import time
import requests
from PIL import Image

seed(123)

In [2]:
global_path = '/data/liufengyuan/NLPinFinance/'

### Download data

In [3]:
# 创建图片文件夹路径
if not os.path.exists(global_path + 'Data'):
    os.mkdir(global_path + 'Data')

In [4]:
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_AMAZON_FASHION.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Arts_Crafts_and_Sewing.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Automotive.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Books.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_CDs_and_Vinyl.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Cell_Phones_and_Accessories.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Clothing_Shoes_and_Jewelry.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Digital_Music.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Electronics.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Grocery_and_Gourmet_Food.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Home_and_Kitchen.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Industrial_and_Scientific.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Kindle_Store.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Movies_and_TV.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Office_Products.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Patio_Lawn_and_Garden.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Pet_Supplies.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Sports_and_Outdoors.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Tools_and_Home_Improvement.json.gz --no-check-certificate
!wget -P /data/liufengyuan/NLPinFinance/Data https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Toys_and_Games.json.gz --no-check-certificate

--2023-02-16 16:52:58--  https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_AMAZON_FASHION.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 33023658 (31M) [application/x-gzip]
Saving to: ‘/data/liufengyuan/NLPinFinance/Data/meta_AMAZON_FASHION.json.gz’

meta_AMAZON_FASHION 100%[===================>]  31.49M  2.32MB/s    in 12s     

2023-02-16 16:53:11 (2.67 MB/s) - ‘/data/liufengyuan/NLPinFinance/Data/meta_AMAZON_FASHION.json.gz’ saved [33023658/33023658]

--2023-02-16 16:53:11--  https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Arts_Crafts_and_Sewing.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
  Unable to locally verify the issuer's authority.

### Unzip files

In [4]:
if not os.path.exists(global_path + 'Unziped_Filtered_Data'):
    os.mkdir(global_path + 'Unziped_Filtered_Data')

In [5]:
filenames = os.listdir(global_path + 'Data')
total = 0
for name in filenames:

    data = {}
    filename_path = os.path.join(global_path + 'Data',name)
    category = re.findall('meta_(.*?).json.gz',name)[0]
    data[category] = []

    with gzip.open(filename_path) as f:

        count = 0
        for l in f:
            count += 1
            pre_data = json.loads(l.strip())
            if 'imageURLHighRes' in pre_data.keys() and 'feature' in pre_data.keys() and 'description' in pre_data.keys() and 'title' in pre_data.keys() and 'asin' in pre_data.keys():
                if len(pre_data['imageURLHighRes']) != 0 and len(pre_data['description']) != 0 and len(pre_data['feature']) != 0 and len(pre_data['title']) != 0 and len(pre_data['asin']) != 0:
                    data[category].append({k : pre_data[k] for k in ('asin','title','feature', 'description', 'imageURLHighRes')})

    print(f'{category}中满足条件的共{len(data[category])}条,占比{round(100*len(data[category])/count,2)}%')
    total += len(data[category])
    # 字典保存
    path = os.path.join(global_path + 'Unziped_Filtered_Data',category)
    f_save = open(path + '.pkl', 'wb')
    pickle.dump(data[category], f_save)
    f_save.close() 
print(total)

Tools_and_Home_Improvement中满足条件的共236198条,占比41.33%
AMAZON_FASHION中满足条件的共8310条,占比4.45%
Office_Products中满足条件的共135100条,占比42.83%
Pet_Supplies中满足条件的共108456条,占比52.65%
Industrial_and_Scientific中满足条件的共25819条,占比15.42%
Toys_and_Games中满足条件的共242439条,占比38.25%
Arts_Crafts_and_Sewing中满足条件的共112253条,占比37.05%
Kindle_Store中满足条件的共0条,占比0.0%
CDs_and_Vinyl中满足条件的共71条,占比0.01%
Sports_and_Outdoors中满足条件的共365763条,占比38.01%
Automotive中满足条件的共418846条,占比44.97%
Cell_Phones_and_Accessories中满足条件的共282198条,占比47.82%
Books中满足条件的共966条,占比0.03%
Patio_Lawn_and_Garden中满足条件的共112378条,占比40.21%
Home_and_Kitchen中满足条件的共536316条,占比41.24%
Clothing_Shoes_and_Jewelry中满足条件的共1513427条,占比56.36%
Electronics中满足条件的共420725条,占比53.5%
Movies_and_TV中满足条件的共53条,占比0.03%
Grocery_and_Gourmet_Food中满足条件的共8209条,占比2.86%
Digital_Music中满足条件的共23条,占比0.03%
4527550


### Get a list of Files

In [6]:
filenames = os.listdir(global_path + 'Unziped_Filtered_Data')
filenames

['Cell_Phones_and_Accessories.pkl',
 'Office_Products.pkl',
 'Movies_and_TV.pkl',
 'Patio_Lawn_and_Garden.pkl',
 'Clothing_Shoes_and_Jewelry.pkl',
 'Electronics.pkl',
 'Industrial_and_Scientific.pkl',
 'Kindle_Store.pkl',
 'Digital_Music.pkl',
 'Home_and_Kitchen.pkl',
 'Sports_and_Outdoors.pkl',
 'CDs_and_Vinyl.pkl',
 'Books.pkl',
 'Arts_Crafts_and_Sewing.pkl',
 'Toys_and_Games.pkl',
 'Grocery_and_Gourmet_Food.pkl',
 'AMAZON_FASHION.pkl',
 'Pet_Supplies.pkl',
 'Automotive.pkl',
 'Tools_and_Home_Improvement.pkl']

In [7]:
# Get files information
file_dict = {}
for file in filenames:
    f_read = open(global_path + 'Unziped_Filtered_Data/' + file, 'rb')
    All_data = pickle.load(f_read)
    f_read.close()
    temp_dict = {}

    tot_cap_len = 0
    tot_fea_len = 0
    tot_des_len = 0
    for idx, item in enumerate(All_data):
        tot_cap_len += len(item['title'])
        for fea in item['feature']:
            tot_fea_len += len(fea)
        for fea in item['description']:
            tot_des_len += len(fea)
    print(f"{file}: (total length: {len(All_data)}) has mean title length {tot_cap_len/(1+len(All_data))}, mean feature length {tot_fea_len/(1+len(All_data))}, mean description length {tot_des_len/(1+len(All_data))}")
    temp_dict['length'] = len(All_data)
    temp_dict['title'] = tot_cap_len/(1+len(All_data))
    temp_dict['feature'] = tot_fea_len/(1+len(All_data))
    temp_dict['description'] = tot_des_len/(1+len(All_data))
    file_dict[file[:-4]] = temp_dict

Cell_Phones_and_Accessories.pkl: (total length: 282198) has mean title length 129.80852518967112, mean feature length 472.4708875651579, mean description length 660.0881044936375
Office_Products.pkl: (total length: 135100) has mean title length 87.18961369641971, mean feature length 401.0041598507783, mean description length 580.8472994278354
Movies_and_TV.pkl: (total length: 53) has mean title length 63.148148148148145, mean feature length 338.037037037037, mean description length 901.7962962962963
Patio_Lawn_and_Garden.pkl: (total length: 112378) has mean title length 81.9882362363075, mean feature length 429.0063179063704, mean description length 667.7810178058178
Clothing_Shoes_and_Jewelry.pkl: (total length: 1513427) has mean title length 621.0240711814503, mean feature length 366.00089597919424, mean description length 516.8315830022967
Electronics.pkl: (total length: 420725) has mean title length 108.96757509638101, mean feature length 484.1350546436398, mean description length 

In [8]:
# get the number of qualified data
for file in filenames:
    f_read = open(global_path + 'Unziped_Filtered_Data/' + file, 'rb')
    All_data = pickle.load(f_read)
    f_read.close()

    count = 0

    for idx, item in enumerate(All_data):

        tot_cap_len = len(item['title'])

        if tot_cap_len < file_dict[file[:-4]]['title']:
            tot_fea_len = 0
            for fea in item['feature']:
                tot_fea_len += len(fea)
            
            if tot_fea_len < file_dict[file[:-4]]['feature']:
                tot_des_len = 0
                for des in item['description']:
                    tot_des_len += len(des)

                if tot_des_len < file_dict[file[:-4]]['description']:
                    count+=1
    file_dict[file[:-4]]['length'] = count

In [9]:
sorted(file_dict.items(), key = lambda x: x[1]['length'], reverse=True)

[('Clothing_Shoes_and_Jewelry',
  {'length': 768609,
   'title': 621.0240711814503,
   'feature': 366.00089597919424,
   'description': 516.8315830022967}),
 ('Home_and_Kitchen',
  {'length': 198095,
   'title': 83.04058980043519,
   'feature': 430.47090806370113,
   'description': 581.5831457887779}),
 ('Electronics',
  {'length': 149181,
   'title': 108.96757509638101,
   'feature': 484.1350546436398,
   'description': 880.4408498642822}),
 ('Automotive',
  {'length': 143482,
   'title': 78.71878514111359,
   'feature': 379.6169508197504,
   'description': 725.1025195357732}),
 ('Sports_and_Outdoors',
  {'length': 137210,
   'title': 68.03204798722673,
   'feature': 822.8115232773044,
   'description': 576.3706761737077}),
 ('Cell_Phones_and_Accessories',
  {'length': 95188,
   'title': 129.80852518967112,
   'feature': 472.4708875651579,
   'description': 660.0881044936375}),
 ('Tools_and_Home_Improvement',
  {'length': 84358,
   'title': 87.55753411318422,
   'feature': 403.7027506

### Get and filter top five files

* 'Clothing_Shoes_and_Jewelry',
* 'Home_and_Kitchen',
* 'Electronics',
* 'Automotive',
* 'Sports_and_Outdoors'

In [10]:
new_file_name = ['Clothing_Shoes_and_Jewelry.pkl', 'Home_and_Kitchen.pkl', 'Electronics.pkl', 'Automotive.pkl', 'Sports_and_Outdoors.pkl']

In [11]:
if not os.path.exists( global_path + 'Five_Categories_Data'):
    os.makedirs( global_path + 'Five_Categories_Data' )

In [12]:
for file in new_file_name:
    f_read = open(global_path + 'Unziped_Filtered_Data/' + file, 'rb')
    All_data = pickle.load(f_read)
    f_read.close()

    new_file = []

    for idx, item in enumerate(All_data):

        tot_cap_len = len(item['title'])

        if tot_cap_len < file_dict[file[:-4]]['title']:
            tot_fea_len = 0
            for fea in item['feature']:
                tot_fea_len += len(fea)
            
            if tot_fea_len < file_dict[file[:-4]]['feature']:
                tot_des_len = 0
                for des in item['description']:
                    tot_des_len += len(des)

                if tot_des_len < file_dict[file[:-4]]['description']:
                    new_file.append(item)
    print('Save file: ' + file[:-4])
    json.dump(new_file, open(global_path + 'Five_Categories_Data/' + file[:-4] + '.json', 'w'))

Save file: Clothing_Shoes_and_Jewelry
Save file: Home_and_Kitchen
Save file: Electronics
Save file: Automotive
Save file: Sports_and_Outdoors


### Download Imager

In [13]:
# 创建图片文件夹路径
if not os.path.exists(global_path + 'Image'):
    os.mkdir(global_path + 'Image')

In [14]:
def loading_image(url, number_code):

    # 爬取图片
    headers={
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.42'
    }
        
    img_name = number_code + '.jpg'
    img_data = requests.get(url=url,headers=headers).content
    img_path = global_path + 'Image/' + img_name
    
    with open (img_path,'wb') as fp:
        fp.write(img_data)

In [18]:
filenames = os.listdir(global_path + 'Five_Categories_Data')
print(filenames)

for filename in filenames:

    # 拿到文件名和种类
    #filename = re.findall('meta_(.*?).json.gz',filenames[iter])[0] + '.pkl'
    filepath = global_path + 'Five_Categories_Data/' + filename
    print(filepath)

    # 读取相应保存字典
    All_data = json.load(open(filepath, 'r'))

    print(f"Number {iter}: Loading {len(All_data)} images from {filename} dataset.")
    # 拿到网址列表
    if len(All_data) > 0:
        start = time.time()

        pool = mp.Pool(48)
        for j in range(len(All_data)):
            url = All_data[j]['imageURLHighRes'][0]
            number_code = All_data[j]['asin']
            process = pool.apply_async(loading_image, args = (url, number_code))
        pool.close()
        pool.join()

        print(f'耗时:{time.time() - start}')

['Home_and_Kitchen.json', 'Sports_and_Outdoors.json', 'Clothing_Shoes_and_Jewelry.json', 'Automotive.json', 'Electronics.json']
/data/liufengyuan/NLPinFinance/Five_Categories_Data/Home_and_Kitchen.json
Number <built-in function iter>: Loading 198095 images from Home_and_Kitchen.json dataset.


KeyboardInterrupt: 

### Resize Image

In [19]:
def resize_image(image, size):
    """Resize an image to the given size."""
    return image.resize(size, Image.Resampling.LANCZOS)

def transfer_image(image_dir, image, size, output_dir):
    with open(os.path.join(image_dir, image), 'r+b') as f:
        try:
            with Image.open(f) as img:
                img = resize_image(img, size)
                img.save(os.path.join(output_dir, image), img.format)
        except:
            1

def resize_images(image_dir, output_dir, size):
    """Resize the images in 'image_dir' and save into 'output_dir'."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    images = os.listdir(image_dir)
    num_images = len(images)

    pool = mp.Pool(48)
    for i, image in enumerate(images):
        pool.apply_async(transfer_image, args = (image_dir, image, size, output_dir))
        if i % 1000 == 0:
            print ("[%d/%d] Resized the images and saved into '%s'."
                   %(i, num_images, output_dir))
    pool.close()
    pool.join()

In [20]:
if not os.path.exists( global_path + 'resized/LFY_2014'):
    os.makedirs( global_path + 'resized/LFY_2014' )
    
image_dir = os.path.join( global_path + 'Image')
output_dir = os.path.join( global_path + 'resized/LFY_2014')

image_size = [ 256, 256 ]
resize_images( image_dir, output_dir, image_size )
print(f"Image number: {len(os.listdir(image_dir))}")
print(f"Resized Image number: {len(os.listdir(output_dir))}") # should be 1231230

[0/1231230] Resized the images and saved into '/data/liufengyuan/NLPinFinance/resized/LFY_2014'.
[1000/1231230] Resized the images and saved into '/data/liufengyuan/NLPinFinance/resized/LFY_2014'.
[2000/1231230] Resized the images and saved into '/data/liufengyuan/NLPinFinance/resized/LFY_2014'.
[3000/1231230] Resized the images and saved into '/data/liufengyuan/NLPinFinance/resized/LFY_2014'.
[4000/1231230] Resized the images and saved into '/data/liufengyuan/NLPinFinance/resized/LFY_2014'.
[5000/1231230] Resized the images and saved into '/data/liufengyuan/NLPinFinance/resized/LFY_2014'.
[6000/1231230] Resized the images and saved into '/data/liufengyuan/NLPinFinance/resized/LFY_2014'.
[7000/1231230] Resized the images and saved into '/data/liufengyuan/NLPinFinance/resized/LFY_2014'.
[8000/1231230] Resized the images and saved into '/data/liufengyuan/NLPinFinance/resized/LFY_2014'.
[9000/1231230] Resized the images and saved into '/data/liufengyuan/NLPinFinance/resized/LFY_2014'.
[10

### Seperate Data

In [21]:
for file in new_file_name:
    this_file = file[:-4]
    print(this_file)
    total_dict = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '.json', 'r'))
    
    print(len(total_dict))#, len(data_test), len(data_val))

    if not os.path.exists( global_path + 'Five_Categories_Data/' + this_file):
        os.makedirs( global_path + 'Five_Categories_Data/' + this_file )

    shuffle(total_dict)

    num_val_test = 10000
    if len(total_dict) > 300000:
        num_val_test = 15000
    elif len(total_dict) < 150000:
        num_val_test = 5000
    new_data_val = total_dict[:num_val_test]
    new_data_test= total_dict[num_val_test:2*num_val_test]
    new_data_train = total_dict[2*num_val_test:]
    

    print('Save new_data_val')
    f_save = open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/' + this_file + '_val.json', 'w')
    json.dump(new_data_val, f_save)
    f_save.close()

    print('Save new_data_test')
    f_save = open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/' + this_file + '_test.json', 'w')
    json.dump(new_data_test, f_save)
    f_save.close()

    print('Save new_data_train')
    f_save = open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/' + this_file + '_train.json', 'w')
    json.dump(new_data_train, f_save)
    f_save.close()

    print(f"{this_file}'s train: {len(new_data_train)} test: {len(new_data_test)} val: {len(new_data_val)}")

Clothing_Shoes_and_Jewelry
768609
Save new_data_val
Save new_data_test
Save new_data_train
Clothing_Shoes_and_Jewelry's train: 738609 test: 15000 val: 15000
Home_and_Kitchen
198095
Save new_data_val
Save new_data_test
Save new_data_train
Home_and_Kitchen's train: 178095 test: 10000 val: 10000
Electronics
149181
Save new_data_val
Save new_data_test
Save new_data_train
Electronics's train: 139181 test: 5000 val: 5000
Automotive
143482
Save new_data_val
Save new_data_test
Save new_data_train
Automotive's train: 133482 test: 5000 val: 5000
Sports_and_Outdoors
137210
Save new_data_val
Save new_data_test
Save new_data_train
Sports_and_Outdoors's train: 127210 test: 5000 val: 5000


### Convert data formet to COCO

In [22]:
splits = ['train', 'val', 'test']

In [23]:
# Mimic COCO dataset: save karpathy_split_train.json, karpathy_split_val.json, karpathy_split_test.json into {this_file}/annotations file
for file in new_file_name:
    this_file = file[:-4]
    print(this_file)
    
    if not os.path.exists( global_path + 'Five_Categories_Data/' + this_file + '/annotations/'):
        os.makedirs( global_path + 'Five_Categories_Data/' + this_file + '/annotations/')

    for split in splits:        

        val_data = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/' + this_file + '_' + split + '.json', 'r'))
        val_dict = {'type': 'caption', 'images':[], 'annotations': []}
        val_len = len(val_data)
        for idx, item in enumerate(val_data):
            if len(item['description']) == 1: # one only keep data with only one description
                temp_anno = {}
                temp_anno['image_id'] = item['asin']
                temp_anno['id'] = idx + 3058
                temp_anno['caption'] = item['title']
                temp_anno['feature'] = item['feature']
                temp_anno['description'] = item['description'][0] # get the first description
                val_dict['annotations'].append(temp_anno)

                temp_img = {}
                temp_img['license'] = random.randint(0, 9)
                temp_img['file_name'] = item['asin'] + '.jpg'
                temp_img['id'] = item['asin']
                val_dict['images'].append(temp_img)
        json.dump(val_dict, open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/annotations/' + 'karpathy_split_' + split + '.json', 'w'))
        print(f"Original length is {val_len}.  Output length is {len(val_dict['images'])}. {len(val_dict['images'])/val_len * 100}%")

Clothing_Shoes_and_Jewelry
Original length is 738609.  Output length is 701668. 94.99857163939242%
Original length is 15000.  Output length is 14265. 95.1%
Original length is 15000.  Output length is 14255. 95.03333333333333%
Home_and_Kitchen
Original length is 178095.  Output length is 173213. 97.25876638872512%
Original length is 10000.  Output length is 9723. 97.23%
Original length is 10000.  Output length is 9755. 97.55%
Electronics
Original length is 139181.  Output length is 126692. 91.0267924501189%
Original length is 5000.  Output length is 4543. 90.86%
Original length is 5000.  Output length is 4512. 90.24%
Automotive
Original length is 133482.  Output length is 128257. 96.08561453978814%
Original length is 5000.  Output length is 4809. 96.17999999999999%
Original length is 5000.  Output length is 4795. 95.89999999999999%
Sports_and_Outdoors
Original length is 127210.  Output length is 123006. 96.69522836255011%
Original length is 5000.  Output length is 4846. 96.92%
Original 

### Get features

In [25]:
# Mimic COCO dataset: save karpathy_split_train.json, karpathy_split_val.json, karpathy_split_test.json into {this_file}/feature/annotations file
for file in new_file_name:
    this_file = file[:-4]
    print(this_file)

    if not os.path.exists( global_path + 'Five_Categories_Data/' + this_file + '/feature/annotations/'):
        os.makedirs( global_path + 'Five_Categories_Data/' + this_file + '/feature/annotations/')

    for split in splits:
        val_data = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/annotations/karpathy_split_' + split + '.json', 'r'))
        val_dict = {'type': 'caption', 'images':[], 'annotations': []}
        val_len = len(val_data['annotations'])


        
        for item in val_data['annotations']:
            temp_anno = {}
            temp_anno['image_id'] = item['image_id']
            temp_anno['id'] = item['id']
            temp_anno['caption'] = item['caption']
            temp_anno['description'] = item['description']
            for fea in item['feature']:
                temp_anno['feature'] = fea
                val_dict['annotations'].append(temp_anno)

        val_dict['images'] = val_data['images'].copy()
        
        json.dump(val_dict, open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/feature/annotations/karpathy_split_' + split + '.json', 'w'))
        print(f"Original image length is {len(val_data['images'])}.  Output length is {len(val_dict['images'])}. {len(val_dict['images'])/len(val_data['images']) * 100}%")
        print(f"Original annotation length is {val_len}.  Output length is {len(val_dict['annotations'])}. {len(val_dict['annotations'])/val_len * 100}%")

Clothing_Shoes_and_Jewelry
Original image length is 701668.  Output length is 701668. 100.0%
Original annotation length is 701668.  Output length is 4275677. 609.3589845910031%
Original image length is 14265.  Output length is 14265. 100.0%
Original annotation length is 14265.  Output length is 86942. 609.4777427269541%
Original image length is 14255.  Output length is 14255. 100.0%
Original annotation length is 14255.  Output length is 86736. 608.4601894072256%
Home_and_Kitchen
Original image length is 173213.  Output length is 173213. 100.0%
Original annotation length is 173213.  Output length is 743824. 429.42735245045117%
Original image length is 9723.  Output length is 9723. 100.0%
Original annotation length is 9723.  Output length is 41853. 430.4535637149028%
Original image length is 9755.  Output length is 9755. 100.0%
Original annotation length is 9755.  Output length is 41957. 430.1076371091748%
Electronics
Original image length is 126692.  Output length is 126692. 100.0%
Orig

### Tokenize data and join again

In [26]:
def title_tokenize(text):
    # 定义分割符，该变量作为超参数，可以根据情况删减
    filters = ['#', '\$', '@', '&', '!',
                '\t', '\n', '\x97', '\x96', 
                ',', '\(', '\)','\[','\]','/',
                ';']

    # 运用正则进行分词:
    # 去除一些特殊记号：
    text = re.sub("&quot", " ", text, flags=re.S)
    text = re.sub("=", " ", text, flags=re.S)
    text = re.sub(":", " ", text, flags=re.S)
    text = re.sub("\s:\s", " ", text, flags=re.S)
    text = re.sub("&amp", " ", text, flags=re.S)
    text = re.sub("AT&amp;T", "brand ATT", text, flags=re.S)
    text = re.sub("T-Mobile", "brand tmobile", text, flags=re.S)
    text = re.sub("\.+", " ", text, flags=re.S)
    # 将网页内容如<>以及分隔符全部替代为空格
    text = re.sub("<.*?>", " ", text, flags=re.S)
    # 将n't转换为not
    text = re.sub("n't", " not", text, flags=re.S)
    # 删去单独出现的-
    text = re.sub("\s-+\s", " ", text, flags=re.S)
    text = re.sub("\s-+$", " ", text, flags=re.S)
    text = re.sub("^-+\s", " ", text, flags=re.S)
    # 将百分数转化：
    text = re.sub("([0-9]+)%", "\g<1> % ", text, flags=re.S)
    # 删除句号并不影响小数
    text = re.sub("\. ", "  ", text, flags=re.S)
    text = re.sub("\.$", "  ", text, flags=re.S)
    text = re.sub("|".join(filters), "  ", text, flags=re.S)

    # 将*删去：
    text = re.sub(" \*([0-9a-zA-Z]+) ", " \g<1> ", text, flags=re.S)
    text = re.sub("\*([0-9a-zA-Z]+) ", "\g<1> ", text, flags=re.S)
    text = re.sub(" \*([0-9a-zA-Z]+)", " \g<1>", text, flags=re.S)

    text = re.sub(" ([0-9a-zA-Z]+)\* ", " \g<1> ", text, flags=re.S)
    text = re.sub("([0-9a-zA-Z]+)\* ", "\g<1> ", text, flags=re.S)
    text = re.sub(" ([0-9a-zA-Z]+)\*", " \g<1>", text, flags=re.S)

    text = re.sub("\*([0-9a-zA-Z]+)\* ", "\g<1> ", text, flags=re.S)
    text = re.sub(" \*([0-9a-zA-Z]+)\* ", " \g<1> ", text, flags=re.S)  
    text = re.sub(" \*([0-9a-zA-Z]+)\*", " \g<1>", text, flags=re.S)
    
    # 将加号单独列出：
    text = re.sub("(.)\+(.)", "\g<1> + \g<2>", text, flags=re.S)
    # 将不是型号且有-和单词的列出：
    text = re.sub("\s([a-zA-Z]+)-([a-zA-Z]+)\s", " \g<1> \g<2> ", text, flags=re.S)
    text = re.sub("\s([a-zA-Z]+)-([a-zA-Z]+)-([a-zA-Z]+)\s", " \g<1> \g<2> \g<3> ", text, flags=re.S)

    text = re.sub("\s-+([0-9a-zA-Z]+)\s", " \g<1> ", text, flags=re.S)
    text = re.sub("\s([0-9a-zA-Z]+)-+\s", " \g<1> ", text, flags=re.S)
    text = re.sub("\s([0-9a-zA-Z]+)-+$", " \g<1>", text, flags=re.S)
    text = re.sub("\s-+([0-9a-zA-Z]+)", " \g<1>", text, flags=re.S)
    # text = re.sub("-([a-zA-Z]+)\s", "\g<1> ", text, flags=re.S)
    # text = re.sub("\s-([a-zA-Z]+)", " \g<1>", text, flags=re.S)
    # 将分隔符全部替代为空格
    # text = re.sub("|".join(filters), " ", text, flags=re.S)
    # 运用strip()返回移除字符串头尾指定的字符（默认为空格或换行符）生成的新字符串:并转换为小写：
    result = [i.strip().lower() for i in text.split()]
    return result

In [27]:
# use our own way to tokenize and join sentences (prepocess text files)
for file in new_file_name:
    this_file = file[:-4]
    print(this_file)

    if not os.path.exists( global_path + 'Five_Categories_Data/' + this_file + '/tokenized/annotations/'):
        os.makedirs( global_path + 'Five_Categories_Data/' + this_file + '/tokenized/annotations/')

    for split in splits:
        val_data = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/annotations/karpathy_split_' + split + '.json', 'r'))
        val_dict = {'type': 'caption', 'images':[], 'annotations': []}
        val_len = len(val_data['annotations'])
        print(split)
        
        count = 0
        for item in val_data['annotations']:
            count+=1
            temp_anno = {}
            temp_anno['image_id'] = item['image_id']
            temp_anno['id'] = item['id']

            temp_anno['caption'] = ' '.join(title_tokenize(item['caption']))
            temp_anno['description'] = ' '.join(title_tokenize(item['description']))
            temp_anno['feature'] = item['feature']

            val_dict['annotations'].append(temp_anno)
            if count % 5000 == 0:
                print(f"{this_file}'s {split}: " + "[%d/%d] Tokenized the captions." %(count, val_len))

        val_dict['images'] = val_data['images'].copy()
        
        json.dump(val_dict, open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/tokenized/annotations/karpathy_split_' + split + '.json', 'w'))
        print(f"Original image length is {len(val_data['images'])}.  Output length is {len(val_dict['images'])}. {len(val_dict['images'])/len(val_data['images']) * 100}%")
        print(f"Original annotation length is {val_len}.  Output length is {len(val_dict['annotations'])}. {len(val_dict['annotations'])/val_len * 100}%")

Clothing_Shoes_and_Jewelry
train
Clothing_Shoes_and_Jewelry's train: [5000/701668] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [10000/701668] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [15000/701668] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [20000/701668] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [25000/701668] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [30000/701668] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [35000/701668] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [40000/701668] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [45000/701668] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [50000/701668] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [55000/701668] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [60000/701668] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [65000/701668] Tokenized the cap

In [28]:
# use our own way to tokenize and join sentences (prepocess text files)
for file in new_file_name:
    this_file = file[:-4]
    print(this_file)
    
    if not os.path.exists( global_path + 'Five_Categories_Data/' + this_file + '/tokenized/feature/annotations/'):
        os.makedirs( global_path + 'Five_Categories_Data/' + this_file + '/tokenized/feature/annotations/')

    for split in splits:
        val_data = json.load(open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/feature/annotations/karpathy_split_' + split + '.json', 'r'))
        val_dict = {'type': 'caption', 'images':[], 'annotations': []}
        val_len = len(val_data['annotations'])
        
        count = 0
        for item in val_data['annotations']:
            count+=1
            temp_anno = {}
            temp_anno['image_id'] = item['image_id']
            temp_anno['id'] = item['id']

            temp_anno['caption'] = ' '.join(title_tokenize(item['caption']))
            temp_anno['description'] = ' '.join(title_tokenize(item['description']))
            temp_anno['feature'] = ' '.join(title_tokenize(item['feature']))

            val_dict['annotations'].append(temp_anno)
            if count % 10000 == 0:
                print(f"{this_file}'s {split}: " + "[%d/%d] Tokenized the captions." %(count, val_len))

        val_dict['images'] = val_data['images'].copy()
        
        json.dump(val_dict, open('/data/liufengyuan/NLPinFinance/Five_Categories_Data/' + this_file + '/tokenized/feature/annotations/karpathy_split_' + split + '.json', 'w'))
        print(f"Original image length is {len(val_data['images'])}.  Output length is {len(val_dict['images'])}. {len(val_dict['images'])/len(val_data['images']) * 100}%")
        print(f"Original annotation length is {val_len}.  Output length is {len(val_dict['annotations'])}. {len(val_dict['annotations'])/val_len * 100}%")

Clothing_Shoes_and_Jewelry
Clothing_Shoes_and_Jewelry's train: [10000/4275677] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [20000/4275677] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [30000/4275677] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [40000/4275677] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [50000/4275677] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [60000/4275677] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [70000/4275677] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [80000/4275677] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [90000/4275677] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [100000/4275677] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [110000/4275677] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [120000/4275677] Tokenized the captions.
Clothing_Shoes_and_Jewelry's train: [130000/4275677] Token